In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
import datetime as dt

# Data Viz 
import seaborn as sns
import matplotlib.pyplot as plt

# Data Manipulation
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Similarity calculation
from sklearn.metrics.pairwise import cosine_similarity

# Import ML libraries
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

# settings
pd.options.display.max_rows = 100
pd.options.display.max_columns = None

# Math
import math

# Remove warnings
import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/bank-product-recommendation/test_ver2.csv
/kaggle/input/bank-product-recommendation/train_ver2.csv


In [2]:
train = pd.read_csv(filepath_or_buffer='/kaggle/input/bank-product-recommendation/train_ver2.csv')
test = pd.read_csv(filepath_or_buffer='/kaggle/input/bank-product-recommendation/test_ver2.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13647309 entries, 0 to 13647308
Data columns (total 48 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   fecha_dato             object 
 1   ncodpers               int64  
 2   ind_empleado           object 
 3   pais_residencia        object 
 4   sexo                   object 
 5   age                    object 
 6   fecha_alta             object 
 7   ind_nuevo              float64
 8   antiguedad             object 
 9   indrel                 float64
 10  ult_fec_cli_1t         object 
 11  indrel_1mes            object 
 12  tiprel_1mes            object 
 13  indresi                object 
 14  indext                 object 
 15  conyuemp               object 
 16  canal_entrada          object 
 17  indfall                object 
 18  tipodom                float64
 19  cod_prov               float64
 20  nomprov                object 
 21  ind_actividad_cliente  float64
 22  renta           

In [5]:
train.describe(include = 'all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
fecha_dato,13647309,17,2016-05-28,931453,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ncodpers,13647309.0,NaN,NaN,NaN,834904.211501,431565.025784,15889.0,452813.0,931893.0,1199286.0,1553689.0
ind_empleado,13619575,5,N,13610977,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pais_residencia,13619575,118,ES,13553710,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sexo,13619505,2,V,7424252,NaN,NaN,NaN,NaN,NaN,NaN,NaN
age,13647309.0,235.0,23.0,542682.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fecha_alta,13619575,6756,2014-07-28,57389,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ind_nuevo,13619575.0,NaN,NaN,NaN,0.059562,0.236673,0.0,0.0,0.0,0.0,1.0
antiguedad,13647309.0,507.0,0.0,134335.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
indrel,13619575.0,NaN,NaN,NaN,1.178399,4.177469,1.0,1.0,1.0,1.0,99.0


In [6]:
# Check missing values
train.isnull().sum()/train.shape[0] * 100

fecha_dato                0.000000
ncodpers                  0.000000
ind_empleado              0.203220
pais_residencia           0.203220
sexo                      0.203732
age                       0.000000
fecha_alta                0.203220
ind_nuevo                 0.203220
antiguedad                0.000000
indrel                    0.203220
ult_fec_cli_1t           99.818330
indrel_1mes               1.097513
tiprel_1mes               1.097513
indresi                   0.203220
indext                    0.203220
conyuemp                 99.986752
canal_entrada             1.363829
indfall                   0.203220
tipodom                   0.203227
cod_prov                  0.685784
nomprov                   0.685784
ind_actividad_cliente     0.203220
renta                    20.475648
segmento                  1.387585
ind_ahor_fin_ult1         0.000000
ind_aval_fin_ult1         0.000000
ind_cco_fin_ult1          0.000000
ind_cder_fin_ult1         0.000000
ind_cno_fin_ult1    

In [8]:
## Inspect the data sample
train.sample(10)

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
328077,2015-01-28,296856,N,ES,H,59,2001-11-16,0.0,164,1.0,NaN,1.0,A,S,N,NaN,KAB,N,1.0,50.0,ZARAGOZA,1.0,64588.56,02 - PARTICULARES,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3706199,2015-06-28,472840,N,ES,V,48,2004-05-07,0.0,134,1.0,NaN,1.0,A,S,N,NaN,KAT,N,1.0,11.0,CADIZ,1.0,257564.79,01 - TOP,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0
920542,2015-02-28,230497,N,ES,V,55,2001-05-08,0.0,170,1.0,NaN,1.0,A,S,N,NaN,KES,N,1.0,30.0,MURCIA,1.0,290854.50,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
13485847,2016-05-28,996259,N,ES,V,32,2011-12-18,1.0,3,1.0,NaN,1.0,A,S,N,NaN,KHM,N,1.0,35.0,"PALMAS, LAS",1.0,NaN,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
6346745,2015-10-28,1097584,N,ES,V,37,2012-11-08,0.0,35,1.0,NaN,1.0,A,S,N,NaN,KAT,N,1.0,28.0,MADRID,1.0,160015.20,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
9255363,2016-01-28,1318354,N,ES,V,24,2014-09-26,0.0,16,1.0,NaN,1.0,I,S,N,NaN,KHE,N,1.0,25.0,LERIDA,0.0,47868.51,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
9741443,2016-01-28,1094036,N,ES,H,27,2012-11-02,0.0,38,1.0,NaN,1.0,I,S,N,NaN,KHE,N,1.0,6.0,BADAJOZ,0.0,62306.07,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
1042646,2015-02-28,1367401,N,ES,H,27,2014-12-10,0.0,7,1.0,NaN,1.0,I,S,N,NaN,KHL,N,1.0,8.0,BARCELONA,0.0,65869.38,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
11065622,2016-03-28,1331519,N,ES,H,22,2014-10-13,0.0,17,1.0,NaN,1.0,I,S,N,NaN,KHE,N,1.0,11.0,CADIZ,0.0,49995.03,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
4489344,2015-07-28,1302624,N,ES,V,24,2014-09-05,0.0,10,1.0,NaN,1.0,I,S,N,NaN,KFC,N,1.0,28.0,MADRID,0.0,38251.35,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0


In [9]:
train.columns

Index(['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo',
       'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel',
       'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext',
       'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'cod_prov',
       'nomprov', 'ind_actividad_cliente', 'renta', 'segmento',
       'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1'],
      dtype='object')

In [10]:
# 1) Feature Name Transformation for better understanding
col_names = {"ncodpers":"cust_id", "ind_empleado":"emp_index","pais_residencia":"residence",
            "sexo":"sex","fecha_alta":"first_date","ind_nuevo":"new_cust","antiguedad":"seniority",
            "indrel":"is_primary","ult_fec_cli_1t":"last_primary_date","indrel_1mes":"cust_type",
            "tiprel_1mes":"cust_rel_type","indresi":"residence_index","indext":"foreigner_index",
            "conyuemp":"spouse_index","canal_entrada":"channel","cod_prov":"province","nomprov":"province_name",
            "ind_actividad_cliente":"active_index","renta":"income","segmento":"segment"}

train.rename(col_names, axis = 1, inplace = True)
test.rename(col_names, axis = 1, inplace = True)

In [11]:
train.columns

Index(['fecha_dato', 'cust_id', 'emp_index', 'residence', 'sex', 'age',
       'first_date', 'new_cust', 'seniority', 'is_primary',
       'last_primary_date', 'cust_type', 'cust_rel_type', 'residence_index',
       'foreigner_index', 'spouse_index', 'channel', 'indfall', 'tipodom',
       'province', 'province_name', 'active_index', 'income', 'segment',
       'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1'],
      dtype='object')

In [12]:
# 2) Data Type Conversion
# Convert the features into their intuitive types
train.age = pd.to_numeric(train.age, errors='coerce')
train.income = pd.to_numeric(train.income, errors='coerce')
train.seniority = pd.to_numeric(train.seniority, errors='coerce')
train.first_date = pd.to_datetime(train.first_date, errors = 'coerce')
train['fecha_dato'] = pd.to_datetime(train['fecha_dato'])

test.age = pd.to_numeric(test.age, errors='coerce')
test.income = pd.to_numeric(test.income, errors='coerce')
test.seniority = pd.to_numeric(test.seniority, errors='coerce')
test.first_date = pd.to_datetime(test.first_date, errors = 'coerce')

test['fecha_dato'] = pd.to_datetime(test['fecha_dato'])

In [13]:
# 3) Missing values imputation

# For features with missing values percentage < 10%, use the most common factor level to impute
cols = ['emp_index','residence','sex','first_date','new_cust','is_primary',"cust_type","cust_rel_type",
       "province","province_name","active_index","channel","segment"]

for i in cols:
    train.loc[train[i].isnull(), i] = train[i].value_counts().index[0]
    test.loc[test[i].isnull(), i] = test[i].value_counts().index[0]
    
# For features with missing value accounts for over 10%, impute their missing values based on the mean at different dimensions
# For Income, impute the missings using the medians of Customer Segment and Province
train['income'].fillna(train['income'].mean(), inplace = True)
test['income'].fillna(test['income'].mean(), inplace = True)

In [14]:
# Drop the last primary date and spouse index fields given over 99% missing values
train.drop(['last_primary_date','spouse_index'], axis = 1, inplace = True)
test.drop(['last_primary_date','spouse_index'], axis = 1, inplace = True)

In [15]:
# Take a look at the cleaned data
train.head()

,fecha_dato,cust_id,emp_index,residence,sex,age,first_date,new_cust,seniority,is_primary,cust_type,cust_rel_type,residence_index,foreigner_index,channel,indfall,tipodom,province,province_name,active_index,income,segment,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35.0,2015-01-12,0.0,6.0,1.0,1.0,A,S,N,KHL,N,1.0,29.0,MALAGA,1.0,87218.100000,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23.0,2012-08-10,0.0,35.0,1.0,1.0,I,S,S,KHE,N,1.0,13.0,CIUDAD REAL,0.0,35548.740000,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23.0,2012-08-10,0.0,35.0,1.0,1.0,I,S,N,KHE,N,1.0,13.0,CIUDAD REAL,0.0,122179.110000,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22.0,2012-08-10,0.0,35.0,1.0,1.0,I,S,N,KHD,N,1.0,50.0,ZARAGOZA,0.0,119775.540000,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23.0,2012-08-10,0.0,35.0,1.0,1.0,A,S,N,KHE,N,1.0,50.0,ZARAGOZA,1.0,134254.318238,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0


In [16]:
# 1) Age group: group the age for lifestages: young, adult, senior, elder
age_group = [0, 20, 45, 65, 100]
age_labels = ['young', 'adult', 'senior', 'elder']

train['age_grouped'] = pd.cut(train.age, bins = age_group, labels = age_labels)
test['agg_grouped'] = pd.cut(test.age, bins = age_group, labels = age_labels)

In [17]:
# 2) Income Group: group the incomes at Low, Ordinary, Median-high and High classes at (real case may be more complex)
train_income_group = [0, np.nanpercentile(train['income'], 25), np.nanpercentile(train['income'], 50), np.nanpercentile(train['income'], 75), np.nanpercentile(train['income'], 100)]
test_income_group = [0, np.nanpercentile(test['income'], 25), np.nanpercentile(test['income'], 50), np.nanpercentile(test['income'], 75), np.nanpercentile(test['income'], 100)]
income_labels = ['Low','Ordinary','Median-high','High']

train['income_grouped'] = pd.cut(train['income'], train_income_group, labels = income_labels)
test['income_grouped'] = pd.cut(test['income'], test_income_group, labels = income_labels)

# Model 1: Popularity-based Recommender for Cold Start Phase

In [19]:
# 1) Cold Start Phase: Popularity-based Recommender

# Define the popularity metric: Purchasing Amount as the single metric
popular_data = []

for i in train.columns[22:-2]:
    count_1 = train[i].value_counts().get(1, 0)  # tránh lỗi nếu không có giá trị 1
    freq = round(count_1 / train.shape[0], 2)
    popular_data.append({
        'product': i,
        'sales_volume': count_1,
        'sales_frequency': freq
    })

popular_df = pd.DataFrame(popular_data)
popular_df.sort_values('sales_frequency', inplace=True, ascending=False)
popular_df.reset_index(drop=True, inplace=True)

In [20]:
# Print the Product Recommendation List for Cold Stat phase
popular_df[['product','sales_frequency']]

,product,sales_frequency
0,ind_cco_fin_ult1,0.66
1,ind_ctop_fin_ult1,0.13
2,ind_recibo_ult1,0.13
3,ind_cno_fin_ult1,0.08
4,ind_ecue_fin_ult1,0.08
5,ind_nom_pens_ult1,0.06
6,ind_reca_fin_ult1,0.05
7,ind_nomina_ult1,0.05
8,ind_dela_fin_ult1,0.04
9,ind_ctpp_fin_ult1,0.04


# Model 2: Enhanced Popularity-based Product Recommender with Time Decay Adjustment

In [24]:
from math import exp

popular_df_time = train.iloc[:, [0,1,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45]].copy()

# Tách năm, tháng và tạo cột ngày
popular_df_time['year'] = popular_df_time['fecha_dato'].dt.year
popular_df_time['month'] = popular_df_time['fecha_dato'].dt.month
popular_df_time['day'] = 1

# Tạo cột year_month dưới dạng datetime
popular_df_time['year_month'] = pd.to_datetime(popular_df_time[['year','month','day']])

# Xoá các cột không cần thiết để tránh lỗi groupby().sum()
popular_df_time.drop(['cust_id', 'fecha_dato', 'year', 'month', 'day'], axis=1, inplace=True)

# Groupby theo thời gian và tính tổng các sản phẩm
popular_df_time_g = popular_df_time.groupby('year_month', as_index=False).sum(numeric_only=True)

# Đưa bảng về dạng dài: mỗi dòng là (thời gian, sản phẩm, volume)
popular_df_time_g = pd.melt(popular_df_time_g, id_vars=['year_month'], 
                            var_name='product_name', value_name='sales_volume')
# popular_df_time_g = pd.melt(popular_df_time_g, id_vars=id_columns, var_name='product_name', value_name='sales_volume')

In [25]:
# Function to calculate time decay weight
def time_decay(t, decay_rate):
    return exp(-decay_rate * t)

# Set decay rate and generate popularity-based recommendations with time decay (the selection of rate is arbitary and usually determined by both business and data team based on use cases)
decay_rate = 0.001

# Calculate the months elapsed relative to the most recent date in the dataset
popular_df_time_g['month_elapsed'] = (popular_df_time_g['year_month'].max() - popular_df_time_g['year_month']).dt.days

# Calculate time decay weight for each row
popular_df_time_g['weight'] = popular_df_time_g['month_elapsed'].apply(lambda x: time_decay(x,decay_rate))

# Calculate weighted purchase volume
popular_df_time_g['weighted_volume'] = popular_df_time_g['sales_volume'] * popular_df_time_g['weight']

# Calculate weighted purchase volume
popular_df_time_g['weighted_volume'] = popular_df_time_g['sales_volume'] * popular_df_time_g['weight']

# Group by product_name and calculate the sum of weighted volumes
product_scores = popular_df_time_g.groupby('product_name',as_index = False)['weighted_volume'].sum()

# Sort the product scores in descending order
recommendations = product_scores.sort_values('weighted_volume',ascending=False)
recommendations

,product_name,weighted_volume
2,ind_cco_fin_ult1,7.151619e+06
20,ind_recibo_ult1,1.395679e+06
7,ind_ctop_fin_ult1,1.392299e+06
12,ind_ecue_fin_ult1,9.038044e+05
4,ind_cno_fin_ult1,8.830533e+05
15,ind_nom_pens_ult1,6.467041e+05
16,ind_nomina_ult1,5.952627e+05
19,ind_reca_fin_ult1,5.728198e+05
21,ind_tjcr_fin_ult1,4.796775e+05
8,ind_ctpp_fin_ult1,4.667433e+05


# 2) Memory-based Collaborative Filtering: User-Based and Item-Based Collaborative Filtering

## 2.1) User-based Collaborative Filtering

In [27]:
# 1. In order to avoid RAM limitation, only use 2016 Jan - 2015 May data for training
df_cf = train[train['fecha_dato'].dt.year == 2016]
df_cf = df_cf[df_cf['fecha_dato'].dt.month <= 5]

# 2. Transform the table into User-Item Matrix for similarity calculation
df_cf = df_cf.iloc[:, [0, 1, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]]

if 'fecha_dato' in df_cf.columns:
    df_cf = df_cf.drop(columns=['fecha_dato'])


# Sum the product holding periods for each user (there should be a 0-5 range, 0 indicating not having the product all the 5 months in 2016,
# 5 indicates the having the product all the 5 months).
df_cf = df_cf.groupby("cust_id").sum()
df_cf = df_cf.fillna(0)
df_cf.index = df_cf.index.astype('string')
# The User-Matrix is a M * N matrix (M: number of customers, N: number of products) and it will extend vertically and horizontally when the new customers or products introduced

# 3. Similarity Calculation: use Cosine Similarity to calculate the user similarities and build the similarity matrix
# Given the memory limitation (RAM), only using the top 10,000 customers to build the model. In reality case, more computional resources are expected to be utilized (e.g. cloud computing)
# to help the Similarity Matrix calculation process for all customers 
df_cf_user = df_cf.copy()
df_cf_user = df_cf_user[:10000]

df_cf_user = pd.DataFrame(cosine_similarity(df_cf_user), index = df_cf_user.index.astype('string'), columns = df_cf_user.index.astype('string'))
df_cf_user.shape

(10000, 10000)

In [28]:
# 4. User-based recommendation model
# The cust_id is the customer id we which to recommend product for, sim_rate is minimum similarity rate for other customers to be included to calculate the predicted purchasing volume
# and the top_n is the top number of products should be recommended to the customer.  
def user_based_recommender(cust_id, sim_rate, top_n):
    
    cust_id = str(cust_id)
    
    # Get the customer list with minimum similarity score
    sim_cust = df_cf_user.loc[(df_cf_user[cust_id] >= sim_rate)&(df_cf_user[cust_id] < 1),cust_id]
    print("There are {} customers are similar to the target customer".format(len(sim_cust)))

    # Get the purchased products of the target customer and get the new user-item matrix including only the similar users and the unpurchased products for the customer
    products = df_cf.loc[cust_id,:]
    purchased_products = [i for i in products.index if products[i] == 0]

    new_matrix = df_cf.loc[sim_cust.index, purchased_products]
    new_matrix['sim_score'] = sim_cust

    # Calculate the Predicted Banking Product purchasing amount (0-5) based on the weighted
    for i in new_matrix.columns:
        if i != 'sim_score':
            new_matrix[i] = new_matrix[i] * new_matrix['sim_score']
        else:
            break
    
    # Aggregate the predicted scores and calculate the average based on the sum of similarity scores.
    # The list the recommended products based on the weighted average purchasing volumes (predicted)
    top_item = (new_matrix.sum()/new_matrix.sum()['sim_score'])[:-1].sort_values(ascending = False)[:top_n]\
    
    # Filter out all products with 0 predicted volume (meaning the customer won't have any interest)
    top_item = top_item[top_item > 0]
    
    # Show there is no recommendation if the there is no item included in the list
    if len(top_item) == 0:
        print('There is no recommendation for the customer')
    
    return top_item

In [29]:
# Testing Case 1: cust_id "15929", similarity score min 0.70, top 10 products
user_based_recommender(cust_id = 15929, sim_rate = 0.70, top_n = 10)

There are 255 customers are similar to the target customer


ind_reca_fin_ult1    1.292503
ind_nom_pens_ult1    1.055330
ind_nomina_ult1      0.717253
ind_ctop_fin_ult1    0.658646
ind_fond_fin_ult1    0.613112
ind_plan_fin_ult1    0.553051
ind_cno_fin_ult1     0.498448
ind_dela_fin_ult1    0.478505
ind_hip_fin_ult1     0.097787
ind_cder_fin_ult1    0.018263
dtype: float64

In [30]:
# Testing Case 2: cust_id "16117", similarity score min 0.80, top 5 products
user_based_recommender(cust_id = 16117, sim_rate = 0.70, top_n = 10)

There are 177 customers are similar to the target customer


ind_recibo_ult1      1.116822
ind_ecue_fin_ult1    0.554707
ind_ctop_fin_ult1    0.451120
ind_fond_fin_ult1    0.233580
ind_ctpp_fin_ult1    0.199420
ind_reca_fin_ult1    0.180961
ind_dela_fin_ult1    0.133500
ind_plan_fin_ult1    0.052868
ind_nom_pens_ult1    0.051068
ind_nomina_ult1      0.030427
dtype: float64

In [31]:
# Testing Case 3: cust_id "16117", similarity score min 0.80, top 5 products
user_based_recommender(cust_id = 36060, sim_rate = 0.90, top_n = 10)

There are 75 customers are similar to the target customer


ind_recibo_ult1      0.267043
ind_dela_fin_ult1    0.037160
ind_tjcr_fin_ult1    0.026152
ind_ecue_fin_ult1    0.025658
dtype: float64

From the recommendations above, we can find for each customer, the model will recommend top N products the customer haven't purchased. We can found that most of the product recommended to the customers are having very low predicted purchasing volume (weighted average volume). This is one of the big challenges of collaborative filtering which is Sparsity, meaning that there are many missing values (zeros) in the the user-item matrix due to the users have not purchased or interacted with certain items. This sparsity can make it challenging to accurately predict scores or recommendations for those items.

Besides, in real life scienrios, as the customer population usually grows with rapid speed, it is costly to maintain the storage and computation of the user-item matrix and user similarity matrix.

Therefore, the user-based recommender is not working well under this scienrio.

## 2.1)  Item-based Collaborative Filtering

In [32]:
# 1. Based on the User-Item Matrix above, use Cosine Similarity to calculate the item similarities and build the similarity matrix
# Given the memory limitation (RAM), only using the top 10,000 customers' into to build the model. 
df_cf_item = df_cf.copy()
df_cf_item = df_cf_item[:10000].T

df_cf_item = pd.DataFrame(cosine_similarity(df_cf_item), index = df_cf_item.index.astype('string'), columns = df_cf_item.index.astype('string'))
df_cf_item.shape

(24, 24)

In [33]:
# 2. Item-based recommendation model
# The cust_id is the customer id we which to recommend product for, sim_rate is minimum similarity rate for other customers to be included to calculate the predicted purchasing volume
# and the top_n is the top number of products should be recommended to the customer.  
def item_based_recommender(cust_id, sim_rate, top_n):
    
    cust_id = str(cust_id)
    
    # Select the product the customer purchased mostly (or choose the most recent product purchased)
    df_cf_user_2 = df_cf.T
    top_product = df_cf_user_2[cust_id]
    top_product = top_product[top_product > 0].sort_values(ascending = False).index[0]

    while len(top_product) == 0:
            print("The customer haven't purchased any product during the period")
            break

    # Get the subset of item similarity matrix based on the top 3 purchased prosucts
    item_sim = df_cf_item[top_product]

    # Removew the product the customer purchased and list the Top N recommendations
    recommend_list = item_sim[(item_sim < 1)&(item_sim > 0)].sort_values(ascending = False)[0:top_n]

    while len(recommend_list) == 0:
        print("There is no product recommended")
        break
    
    return recommend_list

In [34]:
# Test Cse 1: cust_id "15906", minimum similarity rate 0.75, top 10 products
item_based_recommender(cust_id = "15906", sim_rate = 0.75, top_n = 10)

ind_cno_fin_ult1     1.000000
ind_nom_pens_ult1    0.839852
ind_nomina_ult1      0.766189
ind_recibo_ult1      0.713549
ind_tjcr_fin_ult1    0.571442
ind_ecue_fin_ult1    0.548664
ind_reca_fin_ult1    0.507433
ind_ctpp_fin_ult1    0.347925
ind_valo_fin_ult1    0.294894
ind_fond_fin_ult1    0.259799
Name: ind_cno_fin_ult1, dtype: float64

In [35]:
# Test Cse 2: cust_id "15907", minimum similarity rate 0.90, top 5 products
item_based_recommender(cust_id = "15907", sim_rate = 0.90, top_n = 5)

ind_cco_fin_ult1     1.000000
ind_ctop_fin_ult1    0.464800
ind_recibo_ult1      0.372657
ind_valo_fin_ult1    0.320389
ind_ecue_fin_ult1    0.318727
Name: ind_cco_fin_ult1, dtype: float64

In [36]:
# Test Cse 2: cust_id "15903", minimum similarity rate 0.90, top 10 products
item_based_recommender(cust_id = "15903", sim_rate = 0.90, top_n = 10)

ind_cco_fin_ult1     1.000000
ind_ctop_fin_ult1    0.464800
ind_recibo_ult1      0.372657
ind_valo_fin_ult1    0.320389
ind_ecue_fin_ult1    0.318727
ind_reca_fin_ult1    0.264723
ind_fond_fin_ult1    0.263221
ind_dela_fin_ult1    0.249394
ind_tjcr_fin_ult1    0.244886
ind_ctpp_fin_ult1    0.230290
Name: ind_cco_fin_ult1, dtype: float64

# 3) Model-based collaborative filter

In [37]:
def matrix_factorization_svd(user_item_matrix, num_factors = 10, num_iterations = 100, learning_rate = 0.01, regularization_rate = 0.01):
    
    num_users, num_items = user_item_matrix.shape
    
    # Initialize user and item matrices
    user_matrix = np.random.rand(num_users, num_factors)
    item_matrix = np.random.rand(num_items, num_factors)
    matrix = user_item_matrix.values
    
    # Perform Gradient Descent updates
    for iteration in range(num_iterations):
        for i in range(num_users):
            for j in range(num_items):
                if matrix[i, j] > 0:
                    
                    error = matrix[i, j] - np.dot(user_matrix[i, :], item_matrix[j, :])
                    user_gradient = learning_rate * (2 * error * item_matrix[j, :] - regularization_rate * user_matrix[i, :])
                    item_gradient = learning_rate * (2 * error * user_matrix[i, :] - regularization_rate * item_matrix[j, :])
                    user_matrix[i, :] += user_gradient
                    item_matrix[j, :] += item_gradient
    
    return user_matrix, item_matrix

In [38]:
# 2. Create the function that generate the product recommendation list based on the user and item matrices:

# Perform the matrix factorization
user_item_matrix = df_cf.iloc[0:10000,:]
user_factors, item_factors = matrix_factorization_svd(user_item_matrix, num_factors = 10, num_iterations = 100, learning_rate = 0.01, regularization_rate = 0.01)

# Add the customer id and product id back to the factorization matrices for recommendation
user_factors = pd.DataFrame(user_factors, index = user_item_matrix.index)
item_factors = pd.DataFrame(item_factors, index = user_item_matrix.columns)

# Define the Function to recommend top N products for a given user
def FM_recommender(cust_id, top_n):
    
    cust_id = str(cust_id)  
    user_scores = np.dot(user_factors.loc[cust_id], item_factors.T)
    user_scores = pd.Series(user_scores, index = user_item_matrix.columns)  
    recommend_list = user_scores.sort_values(ascending = False)[0:top_n]  
    
    return recommend_list  

In [39]:
# Test Case 1: cust_id "15906", top 10 products
FM_recommender(cust_id = 15906, top_n = 10)

ind_ctma_fin_ult1    8.571489
ind_cco_fin_ult1     6.263254
ind_aval_fin_ult1    5.798032
ind_viv_fin_ult1     5.676623
ind_deme_fin_ult1    5.157900
ind_dela_fin_ult1    5.049656
ind_ctpp_fin_ult1    4.986265
ind_cno_fin_ult1     4.903586
ind_ctop_fin_ult1    4.877613
ind_recibo_ult1      4.876869
dtype: float64

In [40]:
# Test Case 2: cust_id "15907", top 10 products
FM_recommender(cust_id = 15907, top_n = 10)

ind_ecue_fin_ult1    5.240845
ind_cno_fin_ult1     5.224144
ind_viv_fin_ult1     5.058032
ind_deco_fin_ult1    5.033536
ind_deme_fin_ult1    5.023326
ind_fond_fin_ult1    5.021813
ind_plan_fin_ult1    5.015308
ind_ctop_fin_ult1    5.013321
ind_hip_fin_ult1     5.012775
ind_reca_fin_ult1    5.010810
dtype: float64

In [42]:
# Test Case 3: cust_id "15903", top 10 products
FM_recommender(cust_id = 15903, top_n = 10)

ind_valo_fin_ult1    5.810983
ind_recibo_ult1      5.582813
ind_dela_fin_ult1    5.501562
ind_deme_fin_ult1    5.352845
ind_reca_fin_ult1    5.214305
ind_plan_fin_ult1    5.181358
ind_ecue_fin_ult1    5.116864
ind_deco_fin_ult1    5.085539
ind_cco_fin_ult1     5.073155
ind_pres_fin_ult1    5.024287
dtype: float64

# 4. GBDT + LR

In [43]:
# 1. Data Preparison

### Only use the 2016-05-28 data as training input, and use the top 10000 customer's records to enhance the training performance
train_gbdt = train[train['fecha_dato'] == pd.to_datetime('2016-05-28')]
train_gbdt = train_gbdt[0:10000]

# Dummify the categorical features and keep the important ones only
product_list = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
                'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
                'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1','ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
                'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1','ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']
feature_list = ['emp_index','sex','new_cust','is_primary','segment','age_grouped','income_grouped']
col_list = feature_list + product_list

train_gbdt = train_gbdt[col_list]
train_gbdt['new_cust'] = train_gbdt['new_cust'].astype('string')
train_gbdt['is_primary'] = train_gbdt['is_primary'].astype('string')
train_gbdt['segment'] = train_gbdt['segment'].replace({'01 - TOP':'VIP','02 - PARTICULARES':'Individuals','03 - UNIVERSITARIO':'Graduated'}).astype('string')
train_gbdt['emp_index'] = train_gbdt['emp_index'].astype('string')
train_gbdt['sex'] = train_gbdt['sex'].astype('string')
train_gbdt['age_grouped'] = train_gbdt['age_grouped'].astype('string')
train_gbdt['income_grouped'] = train_gbdt['income_grouped'].astype('string')

# Given the product list will be used as response variable to be predicted, the train_dummy will be used as customer features to the training process
train_dummy = train_gbdt.select_dtypes(include = 'string')
train_dummy = pd.get_dummies(train_dummy)

In [44]:
# 2. Training Preparison

# 1) Split the data into training and testing datasets (80% training and 20% testing)
X = train_dummy

# 2) Train the GBDT + LR model for each product and predict the customer's probability of choosing each product in the following period
pred = {}

for i in product_list:
    
    # Use the target product column as the target variable
    if train_gbdt[i].nunique() == 2:
        
        y = train_gbdt[i] 
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
    
        # Train the GBDT model
        gbdt_model = GradientBoostingClassifier(n_estimators=300,  # Number of boosting stages
                                           learning_rate=0.1,  # Learning rate
                                           max_depth=3,  # Maximum depth of each tree
                                           random_state=123) 
        gbdt_model.fit(X_train, y_train)
    
        # Generate GBDT features
        gbdt_features = gbdt_model.apply(X_train)[:, :, 0]
    
        # Train the LR model using the GBDT generated features
        lr_model = LogisticRegression(solver='lbfgs', C=1.0, random_state=123)

        lr_model.fit(gbdt_features, y_train)
    
        # Generate LR features
        gbdt_features_test = gbdt_model.apply(X_test)[:, :, 0]
        lr_features = lr_model.predict_proba(gbdt_features_test)[:, 1]
    
        # Store the predicted score for the product
        pred[i] = lr_features
    
    else: 
        pass

In [45]:
# Append the predicted results of the 2000 testing customers into the feature table and generate sample recommendation list
# Could also use the testing dataset as input
pred = pd.DataFrame(pred)
test_gbdt = pd.concat([X_test.reset_index(names = 'cust_id'), pred],axis = 1, ignore_index = True)

col_list = ['cust_id'] + list(X_test.columns) + list(pred.columns)
test_gbdt.columns = col_list
test_gbdt['cust_id'] = test_gbdt['cust_id'].astype('string')

In [46]:
test_gbdt[list(pred.columns)].sample(10)

,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
1039,0.779859,0.000004,0.108905,0.000001,0.000023,0.287733,0.005923,0.002576,1.046965e-07,0.525595,0.194477,0.211965,3.509047e-08,2.449029e-08,0.000004,0.151450,0.015080,0.130306,5.336719e-08,0.001891,0.058494,0.150248
1052,0.598149,0.000030,0.058740,0.000005,0.000199,0.097618,0.047806,0.000005,6.212585e-04,0.020323,0.043487,0.005094,2.857634e-06,2.135152e-05,0.014456,0.031572,0.033304,0.016026,1.194467e-02,0.037264,0.042018,0.115942
241,0.581450,0.002432,0.067852,0.000003,0.005209,0.079386,0.040101,0.000002,1.528224e-03,0.019339,0.067649,0.011502,4.767983e-03,2.457978e-03,0.010636,0.046871,0.045000,0.022030,3.329348e-06,0.033860,0.034128,0.124423
628,0.584535,0.000002,0.082623,0.000002,0.000272,0.114957,0.062661,0.002597,1.970015e-03,0.016699,0.063881,0.011440,2.701917e-06,1.524551e-03,0.005288,0.101874,0.058463,0.021223,1.620040e-02,0.057666,0.064242,0.140023
733,0.584535,0.000002,0.082623,0.000002,0.000272,0.114957,0.062661,0.002597,1.970015e-03,0.016699,0.063881,0.011440,2.701917e-06,1.524551e-03,0.005288,0.101874,0.058463,0.021223,1.620040e-02,0.057666,0.064242,0.140023
559,0.576824,0.000014,0.083978,0.000003,0.000222,0.083605,0.047978,0.001847,4.171376e-03,0.011768,0.068616,0.017652,3.635210e-03,1.974710e-03,0.005104,0.086489,0.050421,0.038565,1.300591e-02,0.056251,0.054667,0.127565
125,0.576824,0.000014,0.083978,0.000003,0.000222,0.083605,0.047978,0.001847,4.171376e-03,0.011768,0.068616,0.017652,3.635210e-03,1.974710e-03,0.005104,0.086489,0.050421,0.038565,1.300591e-02,0.056251,0.054667,0.127565
34,0.598149,0.000030,0.058740,0.000005,0.000199,0.097618,0.047806,0.000005,6.212585e-04,0.020323,0.043487,0.005094,2.857634e-06,2.135152e-05,0.014456,0.031572,0.033304,0.016026,1.194467e-02,0.037264,0.042018,0.115942
1712,0.507563,0.000007,0.018785,0.000002,0.002195,0.126954,0.032066,0.000190,3.798707e-06,0.030019,0.058525,0.018708,3.527196e-08,4.827820e-03,0.000627,0.020984,0.016033,0.011090,4.163954e-07,0.003048,0.018194,0.038646
1080,0.546829,0.000080,0.086525,0.000001,0.002460,0.129870,0.046047,0.000029,4.264243e-03,0.020070,0.068887,0.003109,6.677724e-06,4.455349e-03,0.013267,0.061361,0.032056,0.012461,1.007607e-02,0.068510,0.074050,0.138367


In [47]:
# Testing the Product Recommendation with three sample customers:

def gbdt_product_recommender(df, cust_id, top_n):
    
    cust_id = str(cust_id)
    prod_list = df.loc[df['cust_id'] == cust_id, pred.columns].T
    prod_list.columns = ['pred_score']
    prod_list = prod_list.sort_values(by = 'pred_score', ascending = False)
    
    # Ouput the top N recommended products based on the customer's features. If the probability is lower than 0.5 do not output (the customer would not want this one)
    prod_list = prod_list[prod_list['pred_score'] >= 0.5]
    recommend_list = prod_list[0:top_n]
    
    while len(recommend_list) == 0:
        print("Based on the customer's info, there is no bank product recommended for now")
        break
    
    return recommend_list

In [48]:
# Test Case 1: Customer ID "12718116", Top 10 products
gbdt_product_recommender(df = test_gbdt, cust_id = "12718116", top_n = 10)

,pred_score
ind_cco_fin_ult1,0.513916


In [49]:
# Test Case 1: Customer ID "12719252", Top 10 products
gbdt_product_recommender(df = test_gbdt, cust_id = "12718529", top_n = 10)

,pred_score
ind_cco_fin_ult1,0.546829


In [50]:
# Test Case 3: Customer ID "12723281", Top 10 products
gbdt_product_recommender(df = test_gbdt, cust_id = "12723281", top_n = 10)

,pred_score
ind_cco_fin_ult1,0.594917


From the results, it seems the recommendation is the same for all three customers, and only one product is listed. The model seems do not perform pretty well during the case. More training data (e.g. longer period such as a year), carefully curated features and data transformation should be applied to enhance the prediction power of the model.